### Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
#Read in the csv file
fatal_crash_file = "Resources/fatal_crashes.csv"
fatal_crash_df = pd.read_csv(fatal_crash_file)
fatal_crash_df.head()

C:\Users\damin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20213010,Qld,1,2021,Tuesday,17:00,Multiple,3,No,No,No,70,Major Cities of Australia,Brisbane - East,Redland (C),Sub-arterial Road,No,No,Weekday,Day
1,20215009,WA,1,2021,Saturday,5:55,Single,1,No,No,No,80,NaN,NaN,NaN,NaN,No,No,Weekend,Night
2,20215018,WA,1,2021,Sunday,19:31,Single,1,No,No,No,50,NaN,NaN,NaN,NaN,No,No,Weekend,Night
3,20211012,NSW,1,2021,Thursday,12:35,Single,1,No,No,No,70,Major Cities of Australia,Sydney - South West,Liverpool,National or State Highway,No,No,Weekday,Day
4,20211020,NSW,1,2021,Tuesday,6:03,Multiple,1,No,No,No,80,Major Cities of Australia,Sydney - Outer West and Blue Mountains,Penrith,Arterial Road,No,No,Weekday,Day


### Transform fatal_crash DataFrame

In [3]:
# Create a filtered dataframe from specific columns
fatalcrash_df = ["Crash ID", "State", "Month", "Year", "Dayweek", "Time", "Number Fatalities", "Bus \nInvolvement",	"Heavy Rigid Truck Involvement", "Articulated Truck Involvement", "Speed Limit"]
fatalcrash_transformed= fatal_crash_df[fatalcrash_df].copy()
fatalcrash_transformed.head()

,Crash ID,State,Month,Year,Dayweek,Time,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit
0,20213010,Qld,1,2021,Tuesday,17:00,3,No,No,No,70
1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
3,20211012,NSW,1,2021,Thursday,12:35,1,No,No,No,70
4,20211020,NSW,1,2021,Tuesday,6:03,1,No,No,No,80


In [4]:
#Rename columns to be PostgreSQL compatible
fatalcrash_transformed.columns = ["crash_id", "State", "Month", "Year", "Dayweek", "Time", "Number_Fatalities", "Bus_Involvement",	"Heavy_Rigid_Truck_Involvement", "Articulated_Truck_Involvement", "Speed_Limit"]
fatalcrash_transformed

,crash_id,State,Month,Year,Dayweek,Time,Number_Fatalities,Bus_Involvement,Heavy_Rigid_Truck_Involvement,Articulated_Truck_Involvement,Speed_Limit
0,20213010,Qld,1,2021,Tuesday,17:00,3,No,No,No,70
1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
3,20211012,NSW,1,2021,Thursday,12:35,1,No,No,No,70
4,20211020,NSW,1,2021,Tuesday,6:03,1,No,No,No,80
...,...,...,...,...,...,...,...,...,...,...,...
46886,19892204,Vic,1,1989,Friday,13:20,1,No,No,No,60
46887,19893326,Qld,1,1989,Monday,9:00,1,No,-9,No,60
46888,19892576,Vic,1,1989,Friday,17:15,1,No,No,No,60
46889,19891122,NSW,1,1989,Tuesday,14:10,1,No,-9,No,60


In [5]:
#Filter to have only WA data
fatalcrash_wa = fatalcrash_transformed.loc[fatalcrash_transformed['State'] == "WA"]
fatalcrash_wa

,crash_id,State,Month,Year,Dayweek,Time,Number_Fatalities,Bus_Involvement,Heavy_Rigid_Truck_Involvement,Articulated_Truck_Involvement,Speed_Limit
1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
8,20215007,WA,1,2021,Monday,14:44,1,No,No,No,-9
10,20215010,WA,1,2021,Tuesday,7:05,1,No,No,No,70
11,20215015,WA,1,2021,Saturday,23:08,1,No,No,No,60
...,...,...,...,...,...,...,...,...,...,...,...
46822,19895191,WA,1,1989,Friday,15:45,2,No,-9,No,110
46828,19895188,WA,1,1989,Friday,15:45,1,No,-9,No,110
46833,19895210,WA,1,1989,Monday,16:55,1,No,-9,No,60
46841,19895079,WA,1,1989,Friday,11:45,1,No,-9,No,110


In [6]:
#Replace -9 (N/A) values in dataframe with 'unknown'
fatalcrash_wa = fatalcrash_wa.replace(["-9"],'Unknown')
fatalcrash_wa

,crash_id,State,Month,Year,Dayweek,Time,Number_Fatalities,Bus_Involvement,Heavy_Rigid_Truck_Involvement,Articulated_Truck_Involvement,Speed_Limit
1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
8,20215007,WA,1,2021,Monday,14:44,1,No,No,No,Unknown
10,20215010,WA,1,2021,Tuesday,7:05,1,No,No,No,70
11,20215015,WA,1,2021,Saturday,23:08,1,No,No,No,60
...,...,...,...,...,...,...,...,...,...,...,...
46822,19895191,WA,1,1989,Friday,15:45,2,No,Unknown,No,110
46828,19895188,WA,1,1989,Friday,15:45,1,No,Unknown,No,110
46833,19895210,WA,1,1989,Monday,16:55,1,No,Unknown,No,60
46841,19895079,WA,1,1989,Friday,11:45,1,No,Unknown,No,110


In [7]:
#Replace unknown (N/A) values in speed column with '0' ---> integer column
fatalcrash_wa['Speed_Limit'] = fatalcrash_wa['Speed_Limit'].replace(['Unknown'], 0)
fatalcrash_wa

,crash_id,State,Month,Year,Dayweek,Time,Number_Fatalities,Bus_Involvement,Heavy_Rigid_Truck_Involvement,Articulated_Truck_Involvement,Speed_Limit
1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
8,20215007,WA,1,2021,Monday,14:44,1,No,No,No,0
10,20215010,WA,1,2021,Tuesday,7:05,1,No,No,No,70
11,20215015,WA,1,2021,Saturday,23:08,1,No,No,No,60
...,...,...,...,...,...,...,...,...,...,...,...
46822,19895191,WA,1,1989,Friday,15:45,2,No,Unknown,No,110
46828,19895188,WA,1,1989,Friday,15:45,1,No,Unknown,No,110
46833,19895210,WA,1,1989,Monday,16:55,1,No,Unknown,No,60
46841,19895079,WA,1,1989,Friday,11:45,1,No,Unknown,No,110


In [8]:
#Use list comprehension to change uppercase letters to lowercase
[x.lower() for x in fatalcrash_wa.columns]

['crash_id',
 'state',
 'month',
 'year',
 'dayweek',
 'time',
 'number_fatalities',
 'bus_involvement',
 'heavy_rigid_truck_involvement',
 'articulated_truck_involvement',
 'speed_limit']

In [9]:
#Put new values into dataframe
fatalcrash_wa.columns = [x.lower() for x in fatalcrash_wa.columns]
fatalcrash_wa

,crash_id,state,month,year,dayweek,time,number_fatalities,bus_involvement,heavy_rigid_truck_involvement,articulated_truck_involvement,speed_limit
1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
8,20215007,WA,1,2021,Monday,14:44,1,No,No,No,0
10,20215010,WA,1,2021,Tuesday,7:05,1,No,No,No,70
11,20215015,WA,1,2021,Saturday,23:08,1,No,No,No,60
...,...,...,...,...,...,...,...,...,...,...,...
46822,19895191,WA,1,1989,Friday,15:45,2,No,Unknown,No,110
46828,19895188,WA,1,1989,Friday,15:45,1,No,Unknown,No,110
46833,19895210,WA,1,1989,Monday,16:55,1,No,Unknown,No,60
46841,19895079,WA,1,1989,Friday,11:45,1,No,Unknown,No,110


### Create database connection

In [10]:
connection_string = "postgres:M00shy123@localhost:5432/Road_Fatalities"
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Confirm tables
engine.table_names()

['fatalaties', 'crashes']

### Load DataFrames into database

In [13]:
fatalcrash_wa.to_sql(name='crashes', con=engine, if_exists='append', index=False)

In [14]:
pd.read_sql_query('select * from crashes', con=engine).head()

,id,crash_id,state,month,year,dayweek,time,number_fatalities,bus_involvement,heavy_rigid_truck_involvement,articulated_truck_involvement,speed_limit
0,1,20215009,WA,1,2021,Saturday,5:55,1,No,No,No,80
1,2,20215018,WA,1,2021,Sunday,19:31,1,No,No,No,50
2,3,20215007,WA,1,2021,Monday,14:44,1,No,No,No,0
3,4,20215010,WA,1,2021,Tuesday,7:05,1,No,No,No,70
4,5,20215015,WA,1,2021,Saturday,23:08,1,No,No,No,60
